In [2]:
import numpy as np
import pandas as pd
from numpy import unique

In [3]:
def wrangle(filepath):
    df=pd.read_csv(filepath)
    drop_cols=[col for col in df.columns if "message_id" in col]
    drop_cols.append("customer_id")
    drop_cols.append("client_id")
    drop_cols.append("sent_at")
    drop_cols.append("created_at")
    drop_cols.append("message_number")
    df.drop(columns=drop_cols,inplace=True)
    return df

In [4]:
df = pd.read_csv("communications.csv")
print(df.shape)
print(df.head())

(145941, 15)
                         message_id                       customer_id  \
0  28d82c62db5989d9441351f58738c6a8  f13e966f9dc1e2aa7bffec6c9efa8e11   
1  0f98a1bbf7f2b66f420816a65c8d9b77  f13e966f9dc1e2aa7bffec6c9efa8e11   
2  97f9ed967d2334ce0b4ebfcfc9b5c25c  f13e966f9dc1e2aa7bffec6c9efa8e11   
3  3d0615ca493d69e16265ab688495c134  f13e966f9dc1e2aa7bffec6c9efa8e11   
4  3eb3a03a007bd1adf7482e6d95f7b942  f13e966f9dc1e2aa7bffec6c9efa8e11   

                            sent_at  clicked  converted  \
0  2021-01-04 10:03:16.376000+00:00        0          0   
1  2021-01-12 08:34:09.376000+00:00        0          0   
2  2021-01-19 08:58:11.376000+00:00        0          0   
3  2021-01-28 10:43:04.376000+00:00        0          0   
4  2021-02-04 11:28:56.376000+00:00        0          0   

                          client_id gender customer_country   age  \
0  cfcd208495d565ef66e7dff9f98764da      F              NaN  51.0   
1  cfcd208495d565ef66e7dff9f98764da      F             

In [5]:
counts=df.nunique()
print(counts)

message_id          145941
customer_id           9751
sent_at             145937
clicked                  2
converted                2
client_id                8
gender                   2
customer_country         3
age                     79
created_at            9751
client_name              8
product_type             3
client_country           3
channel                  2
message_number          39
dtype: int64


In [6]:
to_del=[i for i,v in enumerate(counts) if v<=1] #Delete Useless Column
print(df.columns[to_del])
df.drop(df.columns[to_del],axis=1,inplace=True)
counts=df.nunique()
print(counts)

Index([], dtype='object')
message_id          145941
customer_id           9751
sent_at             145937
clicked                  2
converted                2
client_id                8
gender                   2
customer_country         3
age                     79
created_at            9751
client_name              8
product_type             3
client_country           3
channel                  2
message_number          39
dtype: int64


In [7]:
dups=df.duplicated()
print(dups.any())
print(df[dups])


False
Empty DataFrame
Columns: [message_id, customer_id, sent_at, clicked, converted, client_id, gender, customer_country, age, created_at, client_name, product_type, client_country, channel, message_number]
Index: []


In [8]:
df.drop_duplicates(inplace=True)

In [9]:
#Count Missing Values
print(df.isnull().sum())
print(df.isnull().sum()* 100 / len(df))

message_id              0
customer_id             0
sent_at                 0
clicked                 0
converted               0
client_id               0
gender              31676
customer_country    30146
age                 22020
created_at              0
client_name             0
product_type            0
client_country          0
channel                 0
message_number          0
dtype: int64
message_id           0.000000
customer_id          0.000000
sent_at              0.000000
clicked              0.000000
converted            0.000000
client_id            0.000000
gender              21.704661
customer_country    20.656293
age                 15.088289
created_at           0.000000
client_name          0.000000
product_type         0.000000
client_country       0.000000
channel              0.000000
message_number       0.000000
dtype: float64


In [10]:
#Categorial Correlation using Chi-Square
subset = df[["gender","age","clicked","converted","product_type"]]

In [11]:
subset= wrangle("communications.csv")

In [12]:
subset.head()

,clicked,converted,gender,customer_country,age,client_name,product_type,client_country,channel
0,0,0,F,NaN,51.0,SwanPay,BNPL,UK,SMS
1,0,0,F,NaN,51.0,SwanPay,BNPL,UK,SMS
2,0,0,F,NaN,51.0,SwanPay,BNPL,UK,Email
3,0,0,F,NaN,51.0,SwanPay,BNPL,UK,Email
4,0,0,F,NaN,51.0,SwanPay,BNPL,UK,Email


In [68]:
class relevanceattribute:
        def __init__(self,data,targetinput):
            self.data=data
            self.targetinput=targetinput
        def printdata(self):
            print(self.data)
        def ivanalysis(self):
            def calculate_woe_iv(dataset, feature, target):
                lst = []
                for i in range(dataset[feature].nunique()):
                    val = list(dataset[feature].unique())[i]

                    lst.append({
                        'Value': val,
                        'All': dataset[dataset[feature] == val].count()[feature],
                        'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
                        'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
                    })

                dset = pd.DataFrame(lst)
                #print(dset)
                dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
                dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
                dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
                dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
                dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
                iv = dset['IV'].sum()

                dset = dset.sort_values(by='WoE')

                return dset, iv  
            for col in self.data.columns:
                df, iv = calculate_woe_iv(self.data, col, self.targetinput)
                if col == self.targetinput: continue
                else:
                    flag=0
                    if iv<0.02 and flag==1:
                        print("Result: Useless Indicator")
                        flag=1
                    elif iv>=0.02 and iv<0.1:
                        print("Result: Weak Indicator")
                        flag=1
                    elif iv>=0.1 and iv<0.3:
                        print("Result: Medium Indicator")
                        flag=1
                    elif iv>=0.3 and iv<0.5:
                        print("Result: Strong Indicator")
                        flag=1
                    elif iv>=0.5:
                        print("Result: Suspicious Indicator")
                        flag=1
                    if flag==1:
                        print('WoE and IV for column: {}'.format(col))
                        #print(df)
                        print('IV score: {:.2f}'.format(iv))
                        flag=0
                    print('\n')
    

In [70]:
relevanceattribute(subset,'converted').ivanalysis()

Result: Suspicious Indicator
WoE and IV for column: clicked
IV score: 3.00




C:\Users\USER\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)




Result: Medium Indicator
WoE and IV for column: customer_country
IV score: 0.12


Result: Strong Indicator
WoE and IV for column: age
IV score: 0.36


Result: Suspicious Indicator
WoE and IV for column: client_name
IV score: 0.62


Result: Strong Indicator
WoE and IV for column: product_type
IV score: 0.40


Result: Medium Indicator
WoE and IV for column: client_country
IV score: 0.13






In [71]:
#Attribute Relevance Analysis
# IV Guideline
# <0.02 Useless
# 0.02-0.1 Weak
# 0.1-0.3 Medium
# 0.3-0.5 Strong
# >0.5 Too good to be true

relevanceattribute(subset,'clicked').ivanalysis()


C:\Users\USER\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\USER\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Result: Medium Indicator
WoE and IV for column: converted
IV score: 0.14






Result: Medium Indicator
WoE and IV for column: age
IV score: 0.14


Result: Medium Indicator
WoE and IV for column: client_name
IV score: 0.28


Result: Medium Indicator
WoE and IV for column: product_type
IV score: 0.15


Result: Weak Indicator
WoE and IV for column: client_country
IV score: 0.10


Result: Weak Indicator
WoE and IV for column: channel
IV score: 0.10




In [72]:
# IV Guideline
# <0.02 Useless
# 0.02-0.1 Weak
# 0.1-0.3 Medium
# 0.3-0.5 Strong
# >0.5 Too good to be true
relevanceattribute(subset,'channel').ivanalysis()